Строим запрос к нашему сервису

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse
import requests
import json


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [12]:
X_test.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,15505,15216,1,165,61.0,120,80,1,1,0,0,1
1,80282,19815,1,164,54.0,110,80,1,1,0,0,1
2,21224,20563,1,170,69.0,120,80,1,1,0,0,1
3,89571,16625,1,167,57.0,110,70,1,1,0,0,1
4,67392,14451,1,158,110.0,120,80,1,1,0,0,0


In [2]:
# формируем запрос
def send_json(x):
    
    body = {}
    
    for column in x.index:
        body[column] = x[column]
    
    myurl = 'http://127.0.0.1:5000/' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)

    return response.json()['predictions']

Передаем запрос одиночный из наших данных и посмотрим на ответ

In [8]:
response = send_json(X_test.iloc[3,:])
response

0.1798425122400956

Сделаем N запросов и оценим время

In [13]:
N = 50

In [15]:
%%time
predictions = X_test.iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: total: 0 ns
Wall time: 464 ms


Посчитаем метрику

In [16]:
predictions.tail()

45    0.331689
46    0.565001
47    0.409750
48    0.682299
49    0.609212
dtype: float64

In [17]:
y_test.iloc[N-5:N]

,cardio
45,0
46,1
47,0
48,1
49,1


In [18]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.43003597906935237, F-Score=0.807, Precision=0.742, Recall=0.885


In [19]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.8301282051282051